In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ast
import time
import joblib
import warnings
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

warnings.filterwarnings('ignore')

In [ ]:
test_data=pd.read_csv('/content/test_10Percent_Regression.csv')
test_data

,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,DietQuality,SleepQuality,...,CholesterolLDL,CholesterolHDL,CholesterolTriglycerides,UPDRS,MoCA,FunctionalAssessment,DoctorInCharge,WeeklyPhysicalActivity (hr),MedicalHistory,Symptoms
0,3726,59,Female,Caucasian,Bachelor's,28.738206,No,14.648319,1.329546,7.981796,...,84.272917,93.916109,207.577625,157.487485,28.414715,5.728611,DrXXXConfid,06:28,"{'FamilyHistoryParkinsons': 'No', 'TraumaticBr...","{'Tremor': 'No', 'Rigidity': 'No', 'Bradykines..."
1,4394,86,Female,African American,NaN,31.308134,Yes,13.517672,4.297860,9.109908,...,87.815236,79.642990,146.475670,48.020475,22.708773,3.314998,DrXXXConfid,03:58,"{'FamilyHistoryParkinsons': 'No', 'TraumaticBr...","{'Tremor': 'No', 'Rigidity': 'Yes', 'Bradykine..."
2,3844,68,Female,Caucasian,High School,20.051058,No,5.426161,7.721147,6.728971,...,191.658159,54.229505,334.104222,15.852801,26.467741,2.783504,DrXXXConfid,00:21,"{'FamilyHistoryParkinsons': 'Yes', 'TraumaticB...","{'Tremor': 'No', 'Rigidity': 'No', 'Bradykines..."
3,4154,84,Female,Caucasian,High School,36.328988,No,9.905302,7.811894,7.357736,...,80.093508,91.579795,156.965056,44.577815,5.467317,1.475593,DrXXXConfid,02:06,"{'FamilyHistoryParkinsons': 'No', 'TraumaticBr...","{'Tremor': 'Yes', 'Rigidity': 'No', 'Bradykine..."
4,4618,70,Male,Caucasian,High School,35.811746,No,6.084242,4.633972,5.647728,...,189.671900,61.425889,73.722287,183.510691,2.772640,6.745840,DrXXXConfid,08:06,"{'FamilyHistoryParkinsons': 'No', 'TraumaticBr...","{'Tremor': 'No', 'Rigidity': 'No', 'Bradykines..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,3582,89,Male,African American,High School,18.856323,No,7.747015,8.865053,7.188566,...,138.016874,98.268305,245.729237,52.257977,9.438924,8.523396,DrXXXConfid,08:23,"{'FamilyHistoryParkinsons': 'No', 'TraumaticBr...","{'Tremor': 'No', 'Rigidity': 'No', 'Bradykines..."
199,4727,57,Female,Caucasian,High School,17.991364,No,17.581424,1.464666,8.217048,...,128.335495,77.343181,266.721635,191.461646,10.423660,4.555112,DrXXXConfid,07:40,"{'FamilyHistoryParkinsons': 'No', 'TraumaticBr...","{'Tremor': 'No', 'Rigidity': 'No', 'Bradykines..."
200,3337,54,Male,Caucasian,High School,19.192044,No,13.983090,9.825948,6.612138,...,163.912782,76.422620,192.918438,105.968511,5.617022,1.837199,DrXXXConfid,06:51,"{'FamilyHistoryParkinsons': 'No', 'TraumaticBr...","{'Tremor': 'Yes', 'Rigidity': 'No', 'Bradykine..."
201,3177,67,Male,Caucasian,High School,32.575229,No,8.529659,6.449590,4.114073,...,185.842516,67.579188,60.603986,39.975520,14.331113,6.417172,DrXXXConfid,00:50,"{'FamilyHistoryParkinsons': 'No', 'TraumaticBr...","{'Tremor': 'No', 'Rigidity': 'No', 'Bradykines..."


In [ ]:
test_data.columns

Index(['PatientID', 'Age', 'Gender', 'Ethnicity', 'EducationLevel', 'BMI',
       'Smoking', 'AlcoholConsumption', 'DietQuality', 'SleepQuality',
       'SystolicBP', 'DiastolicBP', 'CholesterolTotal', 'CholesterolLDL',
       'CholesterolHDL', 'CholesterolTriglycerides', 'UPDRS', 'MoCA',
       'FunctionalAssessment', 'DoctorInCharge', 'WeeklyPhysicalActivity (hr)',
       'MedicalHistory', 'Symptoms'],
      dtype='object')

In [ ]:
Target=test_data['UPDRS']

In [ ]:
Target

,UPDRS
0,157.487485
1,48.020475
2,15.852801
3,44.577815
4,183.510691
...,...
198,52.257977
199,191.461646
200,105.968511
201,39.975520


In [ ]:
test_data.drop(['UPDRS'], axis=1, inplace=True)

In [ ]:
test_data.isnull().sum()

,0
PatientID,0
Age,0
Gender,0
Ethnicity,0
EducationLevel,35
BMI,0
Smoking,0
AlcoholConsumption,0
DietQuality,0
SleepQuality,0


In [ ]:
categorical_cols = test_data.select_dtypes(include=['object', 'category']).columns
numerical_cols = test_data.select_dtypes(include=['int64', 'float64', 'int32', 'float32']).columns

print(f"Categorical columns: {list(categorical_cols)}")
print(f"Numerical columns: {list(numerical_cols)}")


if len(categorical_cols) > 0:
  cat_imputer = SimpleImputer(strategy='most_frequent')
  test_data[categorical_cols] = cat_imputer.fit_transform(test_data[categorical_cols])

if len(numerical_cols) > 0:
  num_imputer = SimpleImputer(strategy='mean')
  test_data[numerical_cols] = num_imputer.fit_transform(test_data[numerical_cols])

Categorical columns: ['Gender', 'Ethnicity', 'EducationLevel', 'Smoking', 'DoctorInCharge', 'WeeklyPhysicalActivity (hr)', 'MedicalHistory', 'Symptoms']
Numerical columns: ['PatientID', 'Age', 'BMI', 'AlcoholConsumption', 'DietQuality', 'SleepQuality', 'SystolicBP', 'DiastolicBP', 'CholesterolTotal', 'CholesterolLDL', 'CholesterolHDL', 'CholesterolTriglycerides', 'MoCA', 'FunctionalAssessment']


In [ ]:
test_data.isnull().sum()

,0
PatientID,0
Age,0
Gender,0
Ethnicity,0
EducationLevel,0
BMI,0
Smoking,0
AlcoholConsumption,0
DietQuality,0
SleepQuality,0


In [ ]:
test_data.duplicated().sum()

np.int64(0)

In [ ]:
if test_data.duplicated().sum() > 0:
    test_data.drop_duplicates(inplace=True)

In [ ]:
print(test_data.columns)

Index(['PatientID', 'Age', 'Gender', 'Ethnicity', 'EducationLevel', 'BMI',
       'Smoking', 'AlcoholConsumption', 'DietQuality', 'SleepQuality',
       'SystolicBP', 'DiastolicBP', 'CholesterolTotal', 'CholesterolLDL',
       'CholesterolHDL', 'CholesterolTriglycerides', 'MoCA',
       'FunctionalAssessment', 'DoctorInCharge', 'WeeklyPhysicalActivity (hr)',
       'MedicalHistory', 'Symptoms'],
      dtype='object')


In [ ]:
test_data['MedicalHistory'] = test_data['MedicalHistory'].apply(ast.literal_eval)
test_data['Symptoms'] =test_data['Symptoms'].apply(ast.literal_eval)

medical_df = test_data['MedicalHistory'].apply(pd.Series)
symptoms_df = test_data['Symptoms'].apply(pd.Series)

test_data.drop(['MedicalHistory', 'Symptoms'], axis=1, inplace=True)

test_data = pd.concat([test_data, medical_df, symptoms_df], axis=1)
test_data

,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,DietQuality,SleepQuality,...,Diabetes,Depression,Stroke,Tremor,Rigidity,Bradykinesia,PosturalInstability,SpeechProblems,SleepDisorders,Constipation
0,3726.0,59.0,Female,Caucasian,Bachelor's,28.738206,No,14.648319,1.329546,7.981796,...,No,No,No,No,No,No,No,No,Yes,Yes
1,4394.0,86.0,Female,African American,High School,31.308134,Yes,13.517672,4.297860,9.109908,...,No,No,No,No,Yes,No,No,No,No,No
2,3844.0,68.0,Female,Caucasian,High School,20.051058,No,5.426161,7.721147,6.728971,...,No,Yes,No,No,No,No,No,No,No,Yes
3,4154.0,84.0,Female,Caucasian,High School,36.328988,No,9.905302,7.811894,7.357736,...,No,No,No,Yes,No,No,No,No,Yes,No
4,4618.0,70.0,Male,Caucasian,High School,35.811746,No,6.084242,4.633972,5.647728,...,No,No,No,No,No,No,No,Yes,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,3582.0,89.0,Male,African American,High School,18.856323,No,7.747015,8.865053,7.188566,...,No,No,No,No,No,No,No,No,No,No
199,4727.0,57.0,Female,Caucasian,High School,17.991364,No,17.581424,1.464666,8.217048,...,No,No,No,No,No,Yes,No,Yes,No,No
200,3337.0,54.0,Male,Caucasian,High School,19.192044,No,13.983090,9.825948,6.612138,...,No,No,No,Yes,No,No,No,No,Yes,No
201,3177.0,67.0,Male,Caucasian,High School,32.575229,No,8.529659,6.449590,4.114073,...,No,No,No,No,No,No,Yes,No,No,No


In [ ]:
yes_no_columns = [
    'FamilyHistoryParkinsons', 'TraumaticBrainInjury', 'Smoking',
    'Hypertension', 'Diabetes', 'Depression', 'Stroke',
    'Tremor', 'Rigidity', 'Bradykinesia', 'PosturalInstability',
    'SpeechProblems', 'SleepDisorders', 'Constipation'
]

test_data[yes_no_columns] =test_data[yes_no_columns].applymap(lambda x: 1 if x == 'Yes' else 0)
test_data[yes_no_columns].fillna(test_data[yes_no_columns].mode().iloc[0], inplace=True)

In [ ]:
test_data['Gender'] = test_data['Gender'].map({'Female': 0, 'Male': 1})
test_data['Gender'].fillna(test_data['Gender'].mode()[0], inplace=True)


In [ ]:
education_map = {
    'High School': 0,
    "Bachelor's": 1,
    'Higher': 2
}
test_data['EducationLevel'] = test_data['EducationLevel'].map(education_map)
test_data['EducationLevel'].fillna(test_data['EducationLevel'].mode()[0], inplace=True)

In [ ]:
ethnicity_map = {
    'Caucasian': 0,
    'Asian': 1,
    'African American': 2,
    'Other': 3
}
test_data['Ethnicity'] = test_data['Ethnicity'].map(ethnicity_map)
test_data['Ethnicity'].fillna(test_data['Ethnicity'].mode()[0], inplace=True)

In [ ]:
test_data['DoctorInCharge'] = 1
test_data['DoctorInCharge'].fillna(test_data['DoctorInCharge'].mode()[0], inplace=True)

In [ ]:
def convert_time_to_hours(time_str):
    try:
        h, m = map(int, time_str.split(':'))
        return h + m / 60
    except:
        return np.nan

test_data['WeeklyPhysicalActivity (hr)'] =test_data['WeeklyPhysicalActivity (hr)'].apply(convert_time_to_hours)
test_data['WeeklyPhysicalActivity (hr)']

,WeeklyPhysicalActivity (hr)
0,6.466667
1,3.966667
2,0.350000
3,2.100000
4,8.100000
...,...
198,8.383333
199,7.666667
200,6.850000
201,0.833333


In [ ]:
test_data.isna().sum()

,0
PatientID,0
Age,0
Gender,0
Ethnicity,0
EducationLevel,0
BMI,0
Smoking,0
AlcoholConsumption,0
DietQuality,0
SleepQuality,0


In [ ]:
test_data.columns

Index(['PatientID', 'Age', 'Gender', 'Ethnicity', 'EducationLevel', 'BMI',
       'Smoking', 'AlcoholConsumption', 'DietQuality', 'SleepQuality',
       'SystolicBP', 'DiastolicBP', 'CholesterolTotal', 'CholesterolLDL',
       'CholesterolHDL', 'CholesterolTriglycerides', 'MoCA',
       'FunctionalAssessment', 'DoctorInCharge', 'WeeklyPhysicalActivity (hr)',
       'FamilyHistoryParkinsons', 'TraumaticBrainInjury', 'Hypertension',
       'Diabetes', 'Depression', 'Stroke', 'Tremor', 'Rigidity',
       'Bradykinesia', 'PosturalInstability', 'SpeechProblems',
       'SleepDisorders', 'Constipation'],
      dtype='object')

In [ ]:
# corr_matrix = test_data.corr(numeric_only=True)

In [ ]:
# target_corr = corr_matrix['Diagnosis'].sort_values(ascending=False)
# target_corr

In [ ]:
# for i in range(len(Xabs)):
#     if Xabs[i] >= 0.03:
#         targetcolumns.append(Xabs.index[i])

# print(len(targetcolumns))

In [ ]:
targetcolumns =['BMI',
 'Depression',
 'CholesterolTotal',
 'MoCA',
 'SleepQuality',
 'PosturalInstability',
 'CholesterolHDL']

In [ ]:
test_data = test_data[targetcolumns]
test_data

,BMI,Depression,CholesterolTotal,MoCA,SleepQuality,PosturalInstability,CholesterolHDL
0,28.738206,0,166.498450,28.414715,7.981796,0,93.916109
1,31.308134,0,167.327594,22.708773,9.109908,0,79.642990
2,20.051058,1,249.526945,26.467741,6.728971,0,54.229505
3,36.328988,0,150.554112,5.467317,7.357736,0,91.579795
4,35.811746,0,219.972033,2.772640,5.647728,0,61.425889
...,...,...,...,...,...,...,...
198,18.856323,0,257.338398,9.438924,7.188566,0,98.268305
199,17.991364,0,250.359072,10.423660,8.217048,0,77.343181
200,19.192044,0,251.026232,5.617022,6.612138,0,76.422620
201,32.575229,0,256.102412,14.331113,4.114073,1,67.579188


In [ ]:
scalerX = StandardScaler()
scalerX.fit(test_data)

test_data = scalerX.transform(test_data)

scalerY = StandardScaler()
Target = scalerY.fit_transform(Target.values.reshape(-1, 1))

In [ ]:
test_data

array([[ 0.24293225, -0.54128886, -1.32906275, ...,  0.53367559,
        -0.38326578,  1.37497437],
       [ 0.59206097, -0.54128886, -1.30938511, ...,  1.19189325,
        -0.38326578,  0.7805055 ],
       [-0.93723057,  1.84744242,  0.6414085 , ..., -0.19730863,
        -0.38326578, -0.27795451],
       ...,
       [-1.05392899, -0.54128886,  0.6769903 , ..., -0.26547741,
        -0.38326578,  0.64637856],
       [ 0.76419781, -0.54128886,  0.79746058, ..., -1.72302041,
         2.60915548,  0.27805368],
       [-0.00585948,  1.84744242,  1.59716385, ...,  0.71899943,
        -0.38326578,  1.33747809]])

In [ ]:
model = joblib.load('/content/PolyregressionlLassoModel.pkl')
prediction = model.predict(test_data)
prediction_series = pd.Series(prediction, name='prediction')
prediction_series.to_csv('prediction.csv', index=False)
mse = mean_squared_error(Target, prediction)
r2 = r2_score(Target, prediction)
print(f"Mean Squared Error (MSE): {mse}")
print(f"R² Score: {r2}")

Mean Squared Error (MSE): 4.138714682585055
R² Score: -3.1387146825850554


In [ ]:
prediction=pd.read_csv('/content/prediction.csv')
prediction

,prediction
0,-1.781294
1,-1.819151
2,0.699079
3,-1.674267
4,-1.592053
...,...
198,-1.830344
199,-1.903034
200,-1.803128
201,-2.677136


In [ ]:
prediction.isna().sum()

,0
prediction,0
